# Evaluation of BigBench

In [2]:
import os
with open("/home/billxbf/Documents/myks/openai.key", "r") as f:
    keys = f.readlines()
    os.environ["OPENAI_API_KEY"] = keys[0].strip()
with open("/home/billxbf/Documents/myks/serpapi.key", "r") as f:
    keys = f.readlines()
    os.environ["SERPAPI_API_KEY"] = keys[0].strip()

In [3]:
from utils.DataLoader import DataLoader
from utils.Evaluator import Evaluator
from algos.PWS import *
from algos.react import ReactBase
from algos.notool import IO, CoT
from prompts import fewshots

In [4]:
def save_data(dataset, data, save_path):
    dataset["label"] = data["label"]
    dataset["preds"] = data["preds"]
    dataset["em"]  = data["em"]
    dataset["f1"] = data["f1"]
    dataset["acc"] = data["acc"]
    dataset["wall_time"] = data["wall_time"]
    dataset["total_tokens"] = data["total_tokens"]
    dataset["steps"] = data["steps"]
    dataset["tool_cost"] = data["tool_cost"]
    dataset["token_cost"] = data["token_cost"]
    dataset["total_cost"] = data["total_cost"]
    dataset.to_csv(save_path, index=False)
    return dataset

In [5]:
EVAL_LLM = "gpt-3.5-turbo"
EVAL_DATASET = "gsm8k"
SEED = 2024

## Standard IO

In [8]:
dataset = DataLoader(EVAL_DATASET, seed=SEED).load(500)
io = IO(model_name=EVAL_LLM)
eval = Evaluator(task=EVAL_DATASET, dataset=dataset, algo=io)

Found cached dataset gsm8k (/home/billxbf/workspace/PWS/data/gsm8k/gsm8k/main/1.1.0/37bfb08b1d4fcbb01f06b03d9e1ef5f1fcbd4d3af3d08842c50d7305091285ba)


  0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
response, data = eval.run()
df = save_data(dataset, data, "results/gsm8k_io_chat.csv")
response


******************* Start Evaluation *******************



100%|██████████| 500/500 [34:40<00:00,  4.16s/it]


{'avg_em': 0.046,
 'avg_f1': 0.14472137064256022,
 'avg_acc': 0.268,
 'avg_wall_time': 2.2963828892707823,
 'avg_total_tokens': 101.136,
 'avg_total_cost': 0.000202272,
 'avg_steps': 1.0,
 'avg_token_cost': 0.000202272,
 'avg_tool_cost': 0.0}

In [10]:
df.head()

,question,answer,label,preds,em,f1,acc,wall_time,total_tokens,steps,tool_cost,token_cost,total_cost
0,Gary manages two Amazon distribution centers. ...,First find how many packages the second center...,14000,The second center processes 30000 packages per...,False,0.040816,0,6.025388,155,1,0,0.000310,0.000310
1,The square footage of the two bedrooms in the ...,Let M be the number of square feet in Matha's ...,120,Martha's bedroom is 120 square feet.,False,0.285714,1,1.436168,76,1,0,0.000152,0.000152
2,"Thomas, Toby, and Rebecca worked a total of 15...",Toby: 2x-10\nRebecca: (2x-10)-8=2x-18\nTotal:2...,56,What is the value of x? \n\nWe don't know. \n\...,False,0.000000,0,4.695388,130,1,0,0.000260,0.000260
3,Two-thirds of the class have brown eyes. Half ...,There are 6 * 2 = <<6*2=12>>12 students with b...,18,There are 18 students in total.,False,0.285714,1,0.658440,71,1,0,0.000142,0.000142
4,"Jackie spends 8 hours working, 3 hours of exer...",Jackie spends a total of 8 + 8 + 3 = <<8+8+3=1...,5,5 hours.,False,0.666667,1,0.821176,53,1,0,0.000106,0.000106


## CoT

In [8]:
dataset = DataLoader(EVAL_DATASET, seed=SEED).load(500)
cot = CoT(fewshot=fewshots.GSM8K_COT, model_name=EVAL_LLM)
eval = Evaluator(task=EVAL_DATASET, dataset=dataset, algo=cot)

Found cached dataset gsm8k (/home/billxbf/workspace/PWS/data/gsm8k/gsm8k/main/1.1.0/37bfb08b1d4fcbb01f06b03d9e1ef5f1fcbd4d3af3d08842c50d7305091285ba)


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
response, data = eval.run()
df = save_data(dataset, data, "results/gsm8k_cot_chat.csv")
response


******************* Start Evaluation *******************



 61%|██████    | 306/500 [58:25<45:11, 13.98s/it]  

In [ ]:
df.head()

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 ## REACT

In [4]:
dataset = DataLoader("hotpot_qa", seed=2024).load(200)
react = ReactBase(fewshot=fewshots.HOTPOTQA_REACT, model_name="gpt-3.5-turbo", max_iter=8, verbose=False)
eval = Evaluator(task="hotpot_qa", dataset=dataset, algo=react)

Found cached dataset hotpot_qa (/home/billxbf/workspace/PWS/data/hotpot_qa/hotpot_qa/fullwiki/1.0.0/133b9501f892e5193babbad937bee3b4899deb4691ef4d791e6ac0111c875bb5)


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
response, data = eval.run()
df = save_data(dataset, data, "results/hotpot_qa_react_chat.csv")
response


******************* Start Evaluation *******************



  2%|▏         | 3/200 [00:41<48:23, 14.74s/it]/home/billxbf/anaconda3/lib/python3.9/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/billxbf/anaconda3/lib/python3.9/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.c

{'avg_em': 0.32,
 'avg_f1': 0.39618506493506495,
 'avg_acc': 0.385,
 'avg_wall_time': 23.950300869616594,
 'avg_total_tokens': 9795.136363636364,
 'avg_total_cost': 0.01959027272727273,
 'avg_steps': 4.971590909090909,
 'avg_token_cost': 0.01959027272727273,
 'avg_tool_cost': 0.0}

In [9]:
df.head(10)

,question,answer,preds,em,f1,acc,wall_time,total_tokens,steps,tool_cost,token_cost,total_cost
0,In what city and state was the vocalist for E...,"Redwood City, California",nan,False,0.000000,0,NaN,NaN,NaN,NaN,NaN,NaN
1,"What was the birth date of the director of ""Th...","December 28, 1888March",nan,False,0.000000,0,NaN,NaN,NaN,NaN,NaN,NaN
2,What is the name of the cognac house that make...,Jas Hennessy & Co.,Hennessy,False,0.500000,1,8.891754,5092.0,3.0,0.0,0.010184,0.010184
3,The Wisconsin Badgers football team has had a ...,The Iron Horse,the Iron Horse or the Horse,False,0.666667,0,12.183595,7105.0,4.0,0.0,0.014210,0.014210
4,"Which film director is older, Jan Troell or Ag...",Agnès Varda,Jan Troell,False,0.000000,0,9.318587,5263.0,3.0,0.0,0.010526,0.010526
5,"Foshan Lingnan Mingzhu Gymnasium, where Boxing...",2006,2006,True,1.000000,1,7.464730,3284.0,2.0,0.0,0.006568,0.006568
6,In between Parenting and Cook's Illustrated wh...,Parenting,Parenting,True,1.000000,1,10.895709,7210.0,4.0,0.0,0.014420,0.014420
7,"In the year 2016, how many passengers used the...",11.04 million,Agent stopped due to iteration limit or time l...,False,0.000000,0,33.010987,21835.0,9.0,0.0,0.043670,0.043670
8,What was the nationality of Robert Kardashian'...,American,Agent stopped due to iteration limit or time l...,False,0.000000,0,21.680504,14876.0,9.0,0.0,0.029752,0.029752
9,In what Ohio city does the highway which the n...,Cincinnati,Agent stopped due to iteration limit or time l...,False,0.000000,0,30.278475,17950.0,9.0,0.0,0.035900,0.035900


## PWSBase

In [15]:
dataset = DataLoader(EVAL_DATASET, seed=SEED).load(200)
pwsbase = PWS_Base(fewshot=fewshots.HOTPOTQA_PWS_BASE, planner_model=EVAL_LLM, solver_model=EVAL_LLM)
eval = Evaluator(task=EVAL_DATASET, dataset=dataset, algo=pwsbase)

Found cached dataset trivia_qa (/home/billxbf/workspace/PWS/data/trivia_qa/trivia_qa/rc.nocontext/1.2.0/e73c5e47a8704744fa9ded33504b35a6c098661813d1c2a09892eb9b9e9d59ae)


  0%|          | 0/3 [00:00<?, ?it/s]

In [16]:
response, data = eval.run()
df = save_data(dataset, data, "results/trivia_qa_pwsbase_chat.csv")
response


******************* Start Evaluation *******************



  2%|▏         | 4/200 [01:59<1:26:46, 26.57s/it]/home/billxbf/anaconda3/lib/python3.9/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/billxbf/anaconda3/lib/python3.9/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
100%|██████████| 200/200 [1:26:11<00:00, 25.86s/it]   


{'avg_em': 0.55,
 'avg_f1': 0.6333119970396185,
 'avg_acc': 0.735,
 'avg_wall_time': 18.554993919651917,
 'avg_total_tokens': 1387.1919191919192,
 'avg_total_cost': 0.009328020202020204,
 'avg_steps': 3.6666666666666665,
 'avg_token_cost': 0.009328020202020204,
 'avg_tool_cost': 0.0}

In [13]:
df.head(10)

,question,answer,preds,em,f1,acc,wall_time,total_tokens,steps,tool_cost,token_cost,total_cost
0,Noble is a 2014 film written and directed by S...,The Cripple of Inishmaan,Normal People (miniseries).,False,0.0,0,7.235505,1206,4,0.0,0.02412,0.02412
1,Who was the English clergyman and Archbishop o...,Henry Beaufort,Otto von Ziegenhain led the Battle of Tachov.,False,0.0,0,9.427552,1235,5,0.0,0.02470,0.02470
2,Which fictional character in the Harry Potter ...,Rubeus Hagrid,Rubeus Hagrid.,True,1.0,1,5.455595,1460,3,0.0,0.02920,0.02920
3,What Classification is the album by UK band Se...,Trip hop,Britpop.,False,0.0,0,6.557354,1226,4,0.0,0.02452,0.02452
4,What actor from The Imitation Game also starre...,Mark Strong,Benedict Cumberbatch,False,0.0,0,16.167417,2319,6,0.0,0.04638,0.04638


,question,answer,preds,em,f1,acc,wall_time,total_tokens,steps,tool_cost,token_cost,total_cost
0,Noble is a 2014 film written and directed by S...,The Cripple of Inishmaan,Normal People (miniseries).,False,0.000000,0,7.235505,1206,4,0.0,0.02412,0.02412
1,Who was the English clergyman and Archbishop o...,Henry Beaufort,Otto von Ziegenhain led the Battle of Tachov.,False,0.000000,0,9.427552,1235,5,0.0,0.02470,0.02470
2,Which fictional character in the Harry Potter ...,Rubeus Hagrid,Rubeus Hagrid.,True,1.000000,1,5.455595,1460,3,0.0,0.02920,0.02920
3,What Classification is the album by UK band Se...,Trip hop,Britpop.,False,0.000000,0,6.557354,1226,4,0.0,0.02452,0.02452
4,What actor from The Imitation Game also starre...,Mark Strong,Benedict Cumberbatch,False,0.000000,0,16.167417,2319,6,0.0,0.04638,0.04638
5,This 2006 Iowa Hawkeyes football coach played ...,Kirk Ferentz,"Yes, Kirk Ferentz played linebacker for the Un...",False,0.307692,1,9.570546,1998,5,0.0,0.03996,0.03996
6,Ikivo Animator integrates with a software appl...,Adobe Systems,Hypermedia.,False,0.000000,0,4.108220,723,3,0.0,0.01446,0.01446
7,What American actress born in 1912 did the vio...,Marta Eggerth,Mary Pickford.,False,0.000000,0,11.671225,2403,5,0.0,0.04806,0.04806
8,"Which plant has more species, Teucrium or Atal...",Teucrium,Teucrium has more species.,False,0.400000,1,6.937577,1497,4,0.0,0.02994,0.02994
9,When was the British-American journalist and e...,3 November 1949,Unknown.,False,0.000000,0,8.877709,1484,5,0.0,0.02968,0.02968
